In [1]:
from helpers import toWei, fromWei, Market

Here are the market parameters. The rest of the notebook is designed so that if you change these parameters, the rest of the notebook should execute correctly

In [2]:
params = {
    'price_floor_human':    0.05, # ETH per CMT
    'spread':               200,
    'list_reward':          toWei(1, 'CMT'),
    'stake':                toWei(10, 'CMT'),
    'backend_payment':      5, # percent
    'maker_payment':        25, # percent
    # reserve payment:      100 - backend_payment - maker_payment
    'cost_per_byte':        toWei(1e-8, 'ETH'), # 1 ETH for 100MB
}

CREATOR_BLOCK_SIZE =       toWei(2e3, 'CMT')
CREATOR_INITIAL_SUPPORT =  toWei(100, 'ETH')

Create market and initialize

In [3]:
# Convert human readable floor to units used by market codebase
params['price_floor'] = int(params['price_floor_human'] * 1e9) # wei per billion CMT-wei
params['reserve_payment'] = 100 - params['backend_payment'] - params['maker_payment']

m = Market(params)

m.market_total += CREATOR_BLOCK_SIZE
m.reserve += CREATOR_INITIAL_SUPPORT 

Let's run a simple scenario

In [5]:
m.log()
for i in range(1000):
    m.support(1.0)

m.log()
for i in range(510):
    m.withdraw(10)
m.log()

2000.0 CMT
100.0 ETH
Support at 0.15 ETH per CMT
Withdraw at 0.05 ETH per CMT

5220.8 CMT
1100.0 ETH
Support at 0.4714 ETH per CMT
Withdraw at 0.2107 ETH per CMT

120.8 CMT
25.4 ETH
Support at 0.4714 ETH per CMT
Withdraw at 0.2107 ETH per CMT

